In [1]:
import torch
from supervoice_valle import SupervoceNARModel, Tokenizer
from train.dataset import load_sampler
from IPython.display import Audio, display
import matplotlib.pyplot as plt
from vocos import Vocos

In [2]:
vocos = Vocos.from_pretrained("charactr/vocos-encodec-24khz")
tokenizer = Tokenizer("./tokenizer_text.model")
sampler = load_sampler("./external_datasets/libriheavy/libriheavy_cuts_small.jsonl.gz", "./external_datasets/libriheavy-encodec/", 1, tokenizer)
model = SupervoceNARModel()
checkpoint = torch.load("./output/valle-12.pt", map_location = "cpu")
model.load_state_dict(checkpoint['model'])
# model = model.to(torch.float16)
# model = model.to("cuda")
model.eval()
step = checkpoint['step']
print(checkpoint['step'])

/home/steve/miniconda3/envs/gym/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


3000


In [3]:
audio, text = sampler()
print(audio[0], text[0])
encoded = audio[0]
text = text[0]
# id = "./external_datasets/libriheavy-encodec/9774/extermination_american_bison_1503_librivox_64kb_mp3/exterminationamericanbison_12_hornaday_64kb_99"
# with open(id + ".txt", 'r') as file:
#     text = file.read()
#     text = tokenizer.encode(text)
# encoded = torch.load(id + ".pt")

tensor([[876, 126, 126,  ..., 126, 276, 126],
        [ 87, 544, 580,  ..., 200, 168, 200],
        [705, 426, 426,  ..., 893, 705, 705],
        ...,
        [903, 412,  41,  ..., 963, 881, 701],
        [845, 586, 485,  ..., 586, 928, 876],
        [617, 583, 484,  ..., 835, 534, 701]]) tensor([5058,  353,  538,  341,  280, 1206,  968,   15,  662,  189,  288,   71,
          19,   13, 2495,  139,  189,  185, 1337,   71,   13,   52,  209,  264,
        1786,   33,   40,   52,  121,  230, 1402,  248, 2033,  280,  139,  309,
        1910,   56,   25,   13,  250,  241,   52, 3415,   17, 1081, 6719,   12,
          52,  735,   71,   52,   13,  121,  319, 1073,   71,  588,  518,  224,
          28,   52,  209,   78, 1073,  280,  419,  293,  185,  686,  293,  128,
        4109,   31, 2913,   12,  168, 1220, 1679,   71,   42,  729,  217,  585,
          71,  618,   12, 1018,  293,  185,  686,   45,   52,  280,   33, 1182,
          42,   81,  500,  309,   71,   17,  371, 1499,  121,   13,  9

In [4]:
def decode(tokens):
    features = vocos.codes_to_features(tokens)
    bandwidth_id = torch.tensor([2])  # 6 kbps
    return vocos.decode(features, bandwidth_id=bandwidth_id)
display(Audio(data=decode(encoded).cpu(), rate=24000))
print(encoded.shape)

torch.Size([8, 1278])


In [5]:
condition_text = text
condition_audio = encoded[:,:75*3]
audio = encoded[:,75*3:]
predicted = [audio[0]]
print(audio[0])
for i in range(1, 8):
    p = model(
        condition_text = [condition_text], 
        condition_audio = [condition_audio],
        audio = [torch.stack(predicted)],
        codec = [i]
    )

    # Top-3
    p = p[0]
    v, _ = torch.topk(p.unsqueeze(0), 3)
    v = v.squeeze(0)
    p[p < v[:, [-1]]] = -float('Inf')

    # Sample
    p = torch.nn.functional.softmax(p, dim=-1)
    p = torch.multinomial(p, num_samples=1)
    p = p.squeeze(-1)

    print("inference")
    print(p)
    print(audio[i])
    predicted.append(p.cpu())
predicted = torch.stack(predicted)
predicted = torch.cat([condition_audio, predicted], dim = 1)
display(Audio(data=decode(predicted).cpu(), rate=24000))

tensor([879, 353, 860,  ..., 126, 276, 126])
inference
tensor([601, 913, 857,  ..., 544, 544, 518])
tensor([516, 277, 117,  ..., 200, 168, 200])
inference
tensor([424, 424, 974,  ..., 786, 424, 786])
tensor([632, 878, 933,  ..., 893, 705, 705])
inference
tensor([518, 786, 974,  ...,  36, 518, 786])
tensor([ 412,  588,  239,  ...,  916, 1018,  919])
inference
tensor([786, 741, 937,  ..., 622, 786, 673])
tensor([ 22, 625, 807,  ..., 373, 373, 373])
inference
tensor([786, 448, 913,  ..., 986, 786, 622])
tensor([412, 430, 937,  ..., 963, 881, 701])
inference
tensor([448, 939, 786,  ..., 518, 622, 986])
tensor([823, 134, 248,  ..., 586, 928, 876])
inference
tensor([622, 986, 741,  ..., 518, 939, 518])
tensor([ 871, 1019,  721,  ...,  835,  534,  701])


In [6]:
# plt.imshow(model.positional_embedding.weight.cpu().detach().numpy(), cmap='hot', interpolation='nearest')
# plt.show()

In [7]:
# model_t = SupervoceNARModel()
# for i in range(1, 10):
#     checkpoint = torch.load("./output/valle-05." + str(step - i * 1000) + ".pt")
#     model_t.load_state_dict(checkpoint['model'])
#     model_t = model_t.to(torch.float16)
#     model_t = model_t.to("cuda")
#     predicted = [audio[0]]
#     for i in range(1, 8):
#         p = model_t(
#             condition_text = condition_text.to("cuda"), 
#             condition_audio = condition_audio.to("cuda"),
#             audio = audio.to("cuda"),
#             codec = i
#         )
#         p = torch.nn.functional.softmax(p, dim=-1)
#         p = torch.argmax(p, dim=-1)
#         predicted.append(p.cpu())
#     predicted = torch.stack(predicted)
#     predicted = torch.cat([condition_audio, predicted], dim = 1)
#     display(Audio(data=decode(predicted).cpu(), rate=24000))